# Logical circuits creation

In [1]:
L = 8
J = 1
h = 0.05
lamb = 0.5

particle_pair_left_position = 3
particle_pair_length = 1

final_time = 10
layers = 100
measure_every_layers = 1

import z2chain.circs as circs

logical_circs = circs.particle_pair_quench_simulation_circuits(L, J, h, lamb, particle_pair_left_position, particle_pair_length, final_time, layers, measure_every_layers)
# logical_circs[1].draw(output="mpl")

# Device selection

In [2]:
from qiskit_ibm_runtime import QiskitRuntimeService
from cobos.token import ibm_token

channel = "ibm_quantum"
device = "ibm_kyiv"

service = QiskitRuntimeService(channel=channel, token=ibm_token)
backend = service.get_backend(device)

/var/folders/t9/xqv32vzn2pb091tj9_rz1_zh0000gn/T/ipykernel_34042/373737340.py:8: DeprecationWarning: The get_backend() method is deprecated as of qiskit-ibm-runtime 0.24 and will be removed no sooner than 3 months after the release date. Please use backend() instead.
  backend = service.get_backend(device)


# Circuit transpilation

In [3]:
from utils.hexec import transpile

optimization_level = 3

qubits_inds = None
# qubits_inds = [13, 12, 11, 10, 9, 8, 7, 6, 5, 4,
#                3, 2, 1, 0, 14, 18, 19, 20, 33, 39, 
#                38, 37, 52, 56, 57, 58, 59, 60, 53, 41,
#                42, 43, 44, 45, 54, 64, 65, 66, 67, 68,
#                69, 70, 74, 89, 88, 87, 86, 85, 84, 83,
#                92, 102, 103, 104, 111, 122, 121, 120, 119, 118,
#                117, 116, 115, 114]

physical_circuits = transpile(logical_circs, optimization_level, backend, qubits_inds)

# Circuit execution

### Options

In [4]:
from utils.hexec import execdb

default_shots = 4096
optimization_level = 0 # Deactivate error suppression
resilience_level = 0 # Deactivate error mitigation
execution_database = "z2_chain_jobs.json"
zne_mitigation = "none"
measure_mitigation = True

zne_mitigation_dict = {"none": "",
                       "linear": r"linear",
                       "quadratic": r'polynomial_degree_2',
                       "exponential": r"exponential"}

estimator_options = {
    "default_shots": default_shots,
    "optimization_level": optimization_level,
    "resilience_level": resilience_level,
    "resilience": {
        "zne_mitigation": bool(zne_mitigation_dict[zne_mitigation]),
        "measure_mitigation": measure_mitigation
    }
}

if bool(zne_mitigation_dict[zne_mitigation]):
    estimator_options["resilience"]["zne"] = {"extrapolator": zne_mitigation_dict[zne_mitigation]}

job_db = execdb(execution_database)

### Observables to measure

In [5]:
from z2chain.qiskit_observables import local_pauli_z
from functools import partial

observable_name = "Occupation numbers"
observable_generating_funcs = [partial(local_pauli_z, qubit_ind=i) for i in range(2*L-1)]

### Send job to hardware

In [6]:
jobs = job_db.execute_estimator_batch(backend, estimator_options, physical_circuits, observable_generating_funcs, observable_name)

### Load last job with the same options

In [ ]:
jobs = job_db.search_by_params(estimator_options, physical_circuits, observable_name, limit=1, ibmq_service=service)

# Plots

### Quech plot

In [ ]:
from z2chain.plotting import x_t_plot

x_t_plot(jobs)

### Comparison with circuit tensor network simulation

In [ ]:
from z2chain.plotting import save_site_gauge_observable_matrix, load_site_gauge_observable_matrix
from utils.sexec import execute_simulation_estimator_batch
import os

estimator_options = {
    "default_precision": 0
}

mps_simulator_options = {
    "method": "matrix_product_state",
    "matrix_product_state_max_bond_dimension": 256,
    "matrix_product_state_truncation_threshold": 1e-8,
    "max_parallel_threads": 8,
    "max_parallel_experiments": 8
}

filepath = f"Results/tncirc_z2pairquench_maxt_{final_time}_steps_{layers}_L_{L}_J_{J:.04f}_h_{h:.04f}_lamb_{lamb:.04f}_pp_{particle_pair_left_position}_pl_{particle_pair_length}_bd_{mps_simulator_options["matrix_product_state_max_bond_dimension"]}_prec_{estimator_options['default_precision']}.txt"

if os.path.exists(filepath):
    tn_site_gauge_observable_matrix = load_site_gauge_observable_matrix(filepath)
else:
    observables_generating_funcs = [partial(local_pauli_z, qubit_ind=i) for i in range(2*L-1)]
    tn_site_gauge_observable_matrix = execute_simulation_estimator_batch(mps_simulator_options, estimator_options, logical_circs, observables_generating_funcs)
    save_site_gauge_observable_matrix(tn_site_gauge_observable_matrix, filepath)

In [ ]:
from z2chain.plotting import discrepancies_plot

discrepancies_plot(tn_site_gauge_observable_matrix, jobs)